# Peso de la evidencia y valor de la información usando Python

Por: Naren Castellon

El peso de la evidencia (WOE) [Weight of evidence] y el valor de la información (IV) [Information value]son técnicas simples pero poderosas para realizar la transformación y selección de variables. Estos conceptos tienen una gran conexión con la técnica de modelado de regresión logística. Es ampliamente utilizado en la calificación crediticia para medir la separación de clientes buenos y malos.

La fórmula para calcular WOE y IV se proporciona a continuación.

$$WOE=ln(\frac{\%Event}{No Event\%}) $$
y
$$IV=\sum (\%Event - No Event\%) \times ln(\frac{\%Event}{No Event\%}) $$
o simplemente
$$IV=\sum (\%Event - No Event\%) \times WOE $$


## Las ventajas de la transformación WOE son
1. Controla los valores que faltan
2. Maneja valores atípicos
3. La transformación se basa en el valor logarítmico de las distribuciones. Esto está alineado con la función de salida de regresión logística
4. No hay necesidad de variables ficticias
5. Mediante el uso de la técnica de binning adecuada, puede establecer una relación monótona (ya sea aumento o disminución) entre la variable independiente y dependiente.

Además, el valor IV se puede utilizar para seleccionar variables rápidamente.

|Información value| Predicción|
|-----------------|-----------|
|<0.02            |inútil para la predicción|
|0.02 a 0.1       |predictor débil|
|0.1 a 0.3        |predictor medio|
|0.3 a 0.5        |predictor fuerte|
|>0.5             |Sospechoso o demasiado bueno para ser verdad|



Por convención los valores de la estadística IV en la calificación crediticia pueden interpretarse de la siguiente manera.

Si la estadística IV es:
1. Menos de 0.02, entonces el predictor no es útil para el modelado (separando los Bienes de los Malos)
2. 0.02 a 0.1, entonces el predictor solo tiene una relación débil con la relación de probabilidades Bienes/Malos
3. 0.1 a 0.3, entonces el predictor tiene una relación de fuerza media con la relación de probabilidades Bienes/Malos
4. 0.3 a 0.5, entonces el predictor tiene una fuerte relación con la relación de probabilidades Bienes/Malos.
5. $> 0.5$, relación sospechosa

## Puntos importantes
1. El valor de la información aumenta a medida que aumentan los contenedores / grupos para una variable independiente. Tenga cuidado cuando haya más de 20 contenedores, ya que algunos contenedores pueden tener muy pocos eventos y no eventos.
2. El valor de la información no es un método de selección de características óptimas (variables) cuando se está construyendo un modelo de clasificación que no sea la regresión logística binaria (por ejemplo, bosque aleatorio o SVM), ya que las probabilidades de registro condicional (que predecimos en un modelo de regresión logística) están altamente relacionadas con el cálculo del peso de la evidencia. En otras palabras, está diseñado principalmente para el modelo de regresión logística binaria. También piense de esta manera: el bosque aleatorio puede detectar muy bien la relación no lineal, por lo que seleccionar variables a través de Information Value y usarlas en el modelo de bosque aleatorio podría no producir el modelo predictivo más preciso y robusto.

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('bank.csv', sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [ ]:
df.info()

Cambié la variable 'y' a numérica y la nombré como 'objetivo'. Como se mencionó anteriormente, el binning monotónico asegura que se establezca una relación lineal entre la variable independiente y dependiente. En el código, tengo dos funciones `mono_bin()` y `char_bin()`. La función mono_bin se utiliza para variables numéricas y char_bin se utiliza para variables de caracteres. Utilicé la correlación spearman para realizar el binning monótono.

In [4]:
df['y'].value_counts()

no     4000
yes     521
Name: y, dtype: int64

In [5]:
df['target'] = df['y'].apply(lambda x : 1 if x == 'yes' else 0)  # Convert to numeric
df = df.drop('y',axis=1)

La variable 'max_bin' se utiliza para proporcionar el número máximo de contenedores (categorías) para el binning de variables numéricas. Para algunas variables numéricas, la función mono_bin produce solo una categoría mientras se agrupa. Para evitar eso, tengo otra variable llamada 'force_bin' para asegurarme de que al menos produzca 2 categorías.

El cálculo WOE y IV se puede invocar utilizando el siguiente código.

In [6]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

In [7]:
# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

In [8]:
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

In [9]:
def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv) 

In [10]:
final_iv, IV = data_vars(df,df.target)

In [12]:
final_iv.head()

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,age,19,39,2290,259,0.113100,2031,0.886900,0.497121,0.50775,-0.021156,0.000452
1,age,40,87,2231,262,0.117436,1969,0.882564,0.502879,0.49225,0.021363,0.000452
2,job,admin.,admin.,478,58,0.121339,420,0.878661,0.111324,0.10500,0.058488,0.132519
3,job,blue-collar,blue-collar,946,69,0.072939,877,0.927061,0.132438,0.21925,-0.504101,0.132519
4,job,entrepreneur,entrepreneur,168,15,0.089286,153,0.910714,0.028791,0.03825,-0.284088,0.132519


Sobre la base de la información anterior, las variables se pueden seleccionar en función de su poder predictivo.

|Poder de la Predicción|Nombre de la variable|
|----------------------|---------------------|
|inútil para la predicción|default, age|
|predictor débil|campaing, day, education,marital, loan, balance|
|predictor medio|housing, job, previous,pdays, contact|
|predictor fuerte|month, poutcome|
|Sospechoso o demasiado bueno para ser verdad|duration|


In [13]:
IV.sort_values('IV')

,VAR_NAME,IV
5,default,0.000016
0,age,0.000452
4,day,0.004581
2,campaign,0.023342
7,education,0.031812
11,marital,0.040090
10,loan,0.060791
1,balance,0.076208
8,housing,0.106556
9,job,0.132519


### Aplicar valores WOE a las columnas de nuestro DataFrame.

El siguiente código se puede usar para aplicar los valores de WOE a las columnas de nuestro DataFrame.

In [14]:
transform_vars_list = df.columns.difference(['target'])
transform_prefix = 'new_' # deje este valor en blanco si necesita reemplazar los valores de columna originales

In [15]:
transform_vars_list

Index(['age', 'balance', 'campaign', 'contact', 'day', 'default', 'duration',
       'education', 'housing', 'job', 'loan', 'marital', 'month', 'pdays',
       'poutcome', 'previous'],
      dtype='object')

In [17]:
for var in transform_vars_list:
    small_df = final_iv[final_iv['VAR_NAME'] == var]
    transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
    replace_cmd = ''
    replace_cmd1 = ''
    for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
    replace_cmd = replace_cmd + '0'
    replace_cmd1 = replace_cmd1 + '0'
    if replace_cmd != '0':
        try:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd))
        except:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd1))

In [18]:
df['contact'].value_counts()

cellular     2896
unknown      1324
telephone     301
Name: contact, dtype: int64

In [19]:
df['new_contact'].value_counts()

 0.252971    2896
-0.992072    1324
 0.273413     301
Name: new_contact, dtype: int64

In [20]:
small_df = final_iv[final_iv['VAR_NAME'] == 'contact']

In [21]:
small_df

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
31,contact,cellular,cellular,2896,416,0.143646,2480,0.856354,0.798464,0.62000,0.252971,0.247762
32,contact,telephone,telephone,301,44,0.146179,257,0.853821,0.084453,0.06425,0.273413,0.247762
33,contact,unknown,unknown,1324,61,0.046073,1263,0.953927,0.117083,0.31575,-0.992072,0.247762
